In [44]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
main_path = '/content/drive/MyDrive/MyProject/videos/videos/' 
my_path = '/content/drive/MyDrive/MyProject/'
move_seq_path = my_path + 'MoveSeqs/'
holds_seq_path = my_path + 'HoldsSeqs/'

In [46]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense

import numpy as np
import pandas as pd
import json
import itertools

import sklearn as sk
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from random import randint
from numpy import array

import matplotlib.pyplot as plt

In [47]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


Get the pose estimation and move sequence to test the RNN on

In [48]:
worksheet = pd.read_csv(my_path + 'videos.csv')
worksheet = worksheet.iloc[:,1:]
worksheet

,0
0,IMG_1521.MOV_STAB.MOV
1,IMG_1522.MOV_STAB.MOV
2,IMG_1515.MOV_STAB.MOV
3,IMG_1513.MOV_STAB.MOV
4,IMG_1544.MOV_STAB.MOV
...,...
172,IMG_4054.MOV_STAB.MOV
173,IMG_6820.MOV_STAB.MOV
174,IMG_6821.MOV_STAB.MOV
175,IMG_6824.MOV_STAB.MOV


In [49]:
def convert_df_into_array(df):
    """Concatenates all the rows of a dataframe into a big list of strings.
    WARNING: the column names are not registered, so the order has to be implicitly respected"""
    df_list = []

    for i in range(df.shape[0]):
        df_list.append(list(df.iloc[i]))

    df_list = list(itertools.chain.from_iterable(df_list))
    
    return np.array(df_list)

In [50]:
def pad_seq(move_seq, max_size = 100):
    if move_seq.shape[0] > max_size:
        print("Error")
        return 0
    last = np.repeat(['X'], repeats = max_size - move_seq.shape[0], axis = 0)
    
    move_seq = np.hstack([move_seq, last])

    return move_seq

In [51]:
def standardize_df(df, nb_decimals=2):
    return df.round(nb_decimals)

Prepare encoders for the holds sequence and the move sequence

In [52]:
SOS_token = 0
EOS_token = 1

In [53]:
MAX_LENGTH = 45

In [54]:
nb_decimals = 1

def generate_encoders(nb_decimals=2):
    extrs = ['left_hand', 'right_hand', 'left_foot', 'right_foot']
    range = 2
    coords = np.round(np.linspace(-range,range,2*range*(1+10**nb_decimals)), decimals=nb_decimals)
    coords = [f'{a}_{b}' for a in coords for b in coords]

    # move_seq_classes = np.hstack([extrs, coords, ['0']])
    # holds_seq_classes = np.hstack([['X'], coords, ['0']])

    move_seq_classes = [f'{e}_{c}' for e in extrs for c in coords]
    holds_seq_classes = coords

    voc_holds = len(holds_seq_classes)
    voc_move = len(move_seq_classes)

    encode_move_seq = LabelEncoder()
    encode_move_seq.fit(move_seq_classes)
    encode_holds_seq = LabelEncoder()
    encode_holds_seq.fit(holds_seq_classes)
    
    return encode_move_seq, encode_holds_seq, voc_holds, voc_move
    

encode_move_seq, encode_holds_seq, voc_holds, voc_move = generate_encoders(nb_decimals)



In [55]:
# def get_move_vocabulary():
#   extrs = ['left_hand', 'right_hand', 'left_foot', 'right_foot']
#   range = 2
#   coords = np.round(np.linspace(-range,range,2*range*(1+10**nb_decimals)), decimals=nb_decimals)
#   coords = [f'{a}_{b}' for a in coords for b in coords]

#   move_seq_classes = [f'{e}_{c}' for e in extrs for c in coords]

#   res = {}
#   for 

#   return res

In [56]:
def format_seqs(holds_seq, move_seq):

  H = [f'{holds_seq[i]}_{holds_seq[i+1]}' for i in range(0,holds_seq.shape[0],2)]
  

  M = [f'{move_seq[i+2]}_{move_seq[i]}_{move_seq[i+1]}' for i in range(0,move_seq.shape[0],3)]

  return np.array(H), np.array(M)

In [57]:
def prepare_data(sheet):
  """Returns 2 lists of arrays containing all sequences of holds and moves for each climb"""
  move_seqs = []
  holds_seqs = []
  for i in range(sheet.shape[0]):
    if i%10 == 0:
      print(f'Preparing data for video {i}/{sheet.shape[0] - 1}')
    video = sheet.iloc[i,0]
    try:
      holds_seq = pd.read_csv(holds_seq_path + video + '_HOLDS_SEQ.csv', dtype=float)
      move_seq = pd.read_csv(move_seq_path + video + '_MOVE_SEQ.csv')
    except FileNotFoundError:
      continue

    if(move_seq.shape[0] > 4 and holds_seq.shape[0] > 4):
      try:
        move_seq = standardize_df(move_seq.iloc[:,2:], nb_decimals)
        target = convert_df_into_array(move_seq)

        holds_seq = convert_df_into_array(standardize_df(holds_seq.iloc[:,1:], nb_decimals))

        # Format the sequences
        holds_seq, target = format_seqs(holds_seq, target)

        holds_seqs.append(holds_seq)
        move_seqs.append(target)

      except TypeError:
          continue

  return holds_seqs, move_seqs

holds_seqs, move_seqs = prepare_data(worksheet)
print(f'Prepared sequences of size {len(holds_seqs)}')


Preparing data for video 0/176
Preparing data for video 10/176
Preparing data for video 20/176
Preparing data for video 30/176
Preparing data for video 40/176
Preparing data for video 50/176
Preparing data for video 60/176
Preparing data for video 70/176
Preparing data for video 80/176
Preparing data for video 90/176
Preparing data for video 100/176
Preparing data for video 110/176
Preparing data for video 120/176
Preparing data for video 130/176
Preparing data for video 140/176
Preparing data for video 150/176
Preparing data for video 160/176
Preparing data for video 170/176
Prepared sequences of size 124


Add a few possible permutations of holds to make the prediction insensible to the order, and extend the dataset as well

In [58]:
def add_permutations(holds_seqs, move_seqs, nb_perms = 3):
  H_new = []
  M_new = []
  for i in range(len(holds_seqs)):
    for n in range(nb_perms):
      H_new.append(np.random.permutation(holds_seqs[i]))
      M_new.append(move_seqs[i])

  return H_new, M_new

nb_perms = 15
H_new, M_new = add_permutations(holds_seqs, move_seqs, nb_perms = nb_perms)

In [59]:
print(len(holds_seqs), len(move_seqs), len(H_new), len(M_new))

124 124 1860 1860


In [60]:
def encode_lists(holds_seqs, move_seqs):
  """Encode the lists by using the defined encoders. We also append the EOS token at the end"""
  H = []
  M = []

  for h, m in zip(holds_seqs, move_seqs):
    H.append(list(np.hstack([encode_holds_seq.transform(h), EOS_token])))
    M.append(list(np.hstack([encode_move_seq.transform(m), EOS_token])))

  return H, M

H, M = encode_lists(H_new, M_new)


Separate the first 5 videos to keep them for testing

In [61]:
nb_videos_test = 5
H_test, M_test = H[:nb_perms*nb_videos_test], M[:nb_perms*nb_videos_test]
H, M = H[nb_perms*nb_videos_test:], M[nb_perms*nb_videos_test:]

So now we have 2 lists of "sentences", one containing informations on the holds sequences, and the other on the moves sequences

In [62]:
# Define the maximum lenght of the holds and move sequences
n_input = 30 # 30 holds
n_output = 30 # 30 moves

In [63]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [64]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [65]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

### Prepare training data

In [66]:
# Create pairs of (holds_seq, move_seq)
pairs = [(H[i], M[i]) for i in range(len(H))]

In [67]:
def tensorFromSentence(indexes):
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(pair[0])
    target_tensor = tensorFromSentence(pair[1])
    return (input_tensor, target_tensor)

In [68]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0


    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]])

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [69]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [70]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [71]:
def trainIters(encoder, decoder, n_iters, print_every=10, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
                      
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                    decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [72]:
hidden_size = 512
encoder1 = EncoderRNN(voc_holds, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, voc_move, dropout_p=0.1)

trainIters(encoder1, attn_decoder1, n_iters=5000, print_every=100)

0m 38s (- 62m 49s) (100 1%) 4.9010
1m 21s (- 66m 18s) (200 2%) 4.5194
2m 7s (- 68m 53s) (300 3%) 4.6679
2m 46s (- 66m 34s) (400 4%) 3.9820
3m 25s (- 65m 8s) (500 5%) 3.7861
4m 1s (- 63m 1s) (600 6%) 3.3843
4m 40s (- 62m 3s) (700 7%) 3.4109
5m 20s (- 61m 22s) (800 8%) 3.2988
6m 6s (- 61m 49s) (900 9%) 3.0008
6m 51s (- 61m 41s) (1000 10%) 3.2915
7m 35s (- 61m 27s) (1100 11%) 2.8808
8m 15s (- 60m 35s) (1200 12%) 2.9435
8m 57s (- 59m 56s) (1300 13%) 2.7807
9m 41s (- 59m 29s) (1400 14%) 2.4875
10m 23s (- 58m 50s) (1500 15%) 2.5405
11m 9s (- 58m 33s) (1600 16%) 2.3300
11m 56s (- 58m 20s) (1700 17%) 2.3351
12m 44s (- 58m 3s) (1800 18%) 2.0283
13m 36s (- 58m 1s) (1900 19%) 1.9736
14m 21s (- 57m 26s) (2000 20%) 1.7331
15m 14s (- 57m 21s) (2100 21%) 1.8887
15m 58s (- 56m 39s) (2200 22%) 1.4573
16m 46s (- 56m 8s) (2300 23%) 1.4505
17m 32s (- 55m 32s) (2400 24%) 1.4483
18m 15s (- 54m 47s) (2500 25%) 1.2322
19m 3s (- 54m 15s) (2600 26%) 1.3140
19m 59s (- 54m 2s) (2700 27%) 1.1939
20m 51s (- 53m 38s

In [73]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
  '''sentence = hold sequence to translate into move sequence'''
  with torch.no_grad():
      input_tensor = tensorFromSentence(sentence)
      input_length = input_tensor.size()[0]
      encoder_hidden = encoder.initHidden()

      encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

      for ei in range(input_length):
          encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                    encoder_hidden)
          encoder_outputs[ei] += encoder_output[0, 0]

      decoder_input = torch.tensor([[SOS_token]])  # SOS

      decoder_hidden = encoder_hidden

      decoded_words = []
      decoder_attentions = torch.zeros(max_length, max_length)

      for di in range(max_length):
          decoder_output, decoder_hidden, decoder_attention = decoder(
              decoder_input, decoder_hidden, encoder_outputs)
          decoder_attentions[di] = decoder_attention.data
          topv, topi = decoder_output.data.topk(1)
          if topi.item() == EOS_token:
              decoded_words.append('<EOS>')
              break
          else:
              decoded_words.append(encode_move_seq.inverse_transform([topi.item()])[0])

          decoder_input = topi.squeeze().detach()

      return decoded_words, decoder_attentions[:di + 1]

In [74]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', encode_holds_seq.inverse_transform(pair[0]))
        print('=', encode_move_seq.inverse_transform(pair[0]))
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [75]:
evaluateRandomly(encoder1, attn_decoder1)

> ['0.6_0.6' '0.2_0.7' '0.3_0.5' '0.5_0.4' '0.3_0.6' '0.3_0.7' '0.2_0.7'
 '0.4_0.4' '0.6_0.7' '0.2_0.5' '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1'
 '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1']
= ['left_foot_0.6_0.6' 'left_foot_0.2_0.7' 'left_foot_0.3_0.5'
 'left_foot_0.5_0.4' 'left_foot_0.3_0.6' 'left_foot_0.3_0.7'
 'left_foot_0.2_0.7' 'left_foot_0.4_0.4' 'left_foot_0.6_0.7'
 'left_foot_0.2_0.5' 'left_foot_-0.0_-0.1' 'left_foot_-0.0_-0.1'
 'left_foot_-0.0_-0.1' 'left_foot_-0.0_-0.1' 'left_foot_-0.0_-0.1'
 'left_foot_-0.0_-0.1' 'left_foot_-0.0_-0.1']
< left_hand_0.3_0.6 right_hand_0.6_0.6 left_foot_0.2_0.7 right_foot_0.6_0.7 right_hand_0.5_0.4 left_foot_0.2_0.7 left_hand_0.2_0.5 left_foot_0.3_0.7 left_hand_0.3_0.5 left_hand_0.6_0.6 left_hand_0.4_0.4 <EOS>

> ['0.3_0.8' '0.6_0.9' '0.5_0.6' '0.5_0.9' '0.4_0.6' '0.4_0.6' '0.6_0.8'
 '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1'
 '-0.0_-0.1' '-0.0_-0.1' '-0.0_-0.1']
= ['left_foot_0.3_0.8' 'left_foot_0.6_0.9' 'left_foot

In [76]:
def test(encoder, decoder):
  test_indices = [0]
  for i in test_indices:
    df = pd.DataFrame(columns=['x', 'y', 'limb'])
    video = worksheet.iloc[i,0]
    print(video)
    output_words, attentions = evaluate(encoder, decoder, H_test[i])
    print(output_words)
    coords = []
    limb = ''
    entry = {}
    for ind, w in enumerate(output_words):
      if not w == '<EOS>':
        for t in w:
          try:
              coords.append(float(t))
          except ValueError:
              limb += t

        limb = limb.replace('_._.', '')
        entry['x'] = np.round(coords[0] + 0.1*coords[1], decimals=1)
        entry['y'] = np.round(coords[2] + 0.1*coords[3], decimals=1)
        entry['limb'] = limb
        # entry.append(limb)
        df.loc[ind] = entry

        # Reset variables
        coords = []
        limb = ''
        entry = {}
    
    return df

df = test(encoder1, attn_decoder1)
df


IMG_1521.MOV_STAB.MOV
['left_foot_0.4_0.9', 'right_foot_0.4_0.9', 'left_hand_0.4_0.7', 'right_hand_0.4_0.7', 'left_hand_0.5_0.5', 'right_hand_0.4_0.5', 'right_foot_0.5_0.6', 'right_hand_0.4_0.6', 'left_foot_0.5_0.6', 'right_hand_0.5_0.4', 'right_foot_0.7_0.5', 'left_hand_0.5_0.4', '<EOS>']


,x,y,limb
0,0.4,0.9,left_foot
1,0.4,0.9,right_foot
2,0.4,0.7,left_hand
3,0.4,0.7,right_hand
4,0.5,0.5,left_hand
5,0.4,0.5,right_hand
6,0.5,0.6,right_foot
7,0.4,0.6,right_hand
8,0.5,0.6,left_foot
9,0.5,0.4,right_hand


In [78]:
my_path = '/content/drive/MyDrive/MyProject/'
df.to_csv(my_path + 'boulder_2_01_MOVE_SEQ_generated.csv')